# Grid Search for K-Means in Latent Space (+ Comparison to Initial Space)

In [ ]:
import os

from multiprocessing.pool import ThreadPool

import ctypes
from ctypes import *

import numpy as np

from tensorflow.keras.models import load_model

from autoencoder import Autoencoder

from helper_funcs import *

import pandas
pandas.set_option('display.max_rows', None)

from params import get_kmeans_eval_object, get_centroids, convert_to_2d_array, get_stotal, free_centroids, free_kmeans

In [ ]:
models = os.listdir('./models/')

dataset = b'MNIST/input.dat'
query   = b'MNIST/query.dat'

model_to_files = {}
for i, model in enumerate(models):
    normalized_dataset = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_normalized_dataset.dat'
    normalized_query   = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_normalized_query.dat'
    encoded_dataset    = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_encoded_dataset.dat'
    encoded_query      = b'MNIST/' + models[i].removesuffix('.keras').encode() + b'_encoded_query.dat'

    model_to_files.update({models[i] : [normalized_dataset, normalized_query,
                                        encoded_dataset, encoded_query]})

n = 60000

In [ ]:
for model in model_to_files:
    normalized_dataset, normalized_query, encoded_dataset, encoded_query = model_to_files[model]

    model = b'models/' + model.encode()

    # load model
    autoencoder = load_model(model.decode())
    shape = autoencoder.layers[-2].output_shape[1:] # get shape of encoded layer

    # load dataset
    x_train = load_dataset(dataset)
    x_train = x_train.astype('float32') / 255.
    x_test = load_dataset(query)
    x_test = x_test.astype('float32') / 255.
    if len(shape) == 3: # if model type is convolutional
        x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
        x_test = np.reshape(x_test, (len(x_test), 28, 28, 1))
    else:
        x_train = np.reshape(x_train, (len(x_train), 784))
        x_test = np.reshape(x_test, (len(x_test), 784))

    encoded_train = autoencoder.encode(x_train)
    encoded_test = autoencoder.encode(x_test)

    # deflatten encoded datasets
    encoded_train = deflatten_encoded(encoded_train, shape)
    encoded_test = deflatten_encoded(encoded_test, shape)

    # save original datasets normalized
    save_decoded_binary(x_train, normalized_dataset)
    save_decoded_binary(x_test, normalized_query)

    # normalize encoded datasets
    encoded_train = normalize(encoded_train)
    encoded_test = normalize(encoded_test)

    # save encoded datasets
    save_encoded_binary(encoded_train, encoded_dataset)
    save_encoded_binary(encoded_test, encoded_query)

In [ ]:
best_params_lsh = [4, 7, 1, 0.6]            # L, k, limit_queries, window
best_params_hypercube = [67, 3, 1000, 0.42] # M, k, probes, window

rows = []

In [ ]:
def print_results(stotal_lat_init, silouette_lat_init, obj_func):
    print("Total silhouette:", stotal_lat_init)
    print("Silhouette per cluster:", silouette_lat_init)
    print("Objective function:", obj_func)
    print("-------------------------")

def decode_centroids(model, centroids): # decodes centroids and returns them as c array
    model = b'models/' + model.encode()

    autoencoder = load_model(model.decode())
    shape = autoencoder.layers[-2].output_shape[1:] # shape of encoded layer

    centroids = deflatten_encoded(centroids, shape)
    decoded_centroids = autoencoder.decode(centroids)
    decoded_centroids = flatten_encoded(decoded_centroids)

    decoded_centroids = decoded_centroids.astype(np.float64) # cast to float64, equivalent to double in c
    decoded_centroids = decoded_centroids.flatten() # flatten to 1d numpy array
    decoded_centroids = decoded_centroids.ctypes.data_as(ctypes.POINTER(ctypes.c_double)) # convert to c array
    decoded_centroids = convert_to_2d_array(decoded_centroids, 784) # convert to 2d c array, shape: (10, 784)

    return decoded_centroids

def compute_silhouette(model, config):
    kmeans = get_kmeans_eval_object(conf=config)
    centroids, dim = get_centroids(kmeans)

    decoded_centroids = decode_centroids(model, centroids)

    # total silhouette and silhouette per cluster in latent space converted to initial space
    stotal_lat_init, silouette_lat_init, obj_val = get_stotal(config, dim, kmeans, decoded_centroids)

    free_centroids(decoded_centroids)
    free_kmeans(kmeans)

    print_results(stotal_lat_init.value, silouette_lat_init.val, obj_val.value)

    rows.append([model, config['model'].decode(), stotal_lat_init.value, obj_val.value])

    del silouette_lat_init

def run_kmeans_classic(model):
    normalized_dataset, encoded_dataset = model_to_files[model][0], model_to_files[model][2]

    config = {
        'model': b'CLASSIC',
        'vals': [],
        'dataset': normalized_dataset,
        'encoded_dataset': encoded_dataset,
    }
    
    compute_silhouette(model, config)

def run_kmeans_lsh(model):
    normalized_dataset, encoded_dataset = model_to_files[model][0], model_to_files[model][2]

    config = {
        'model': b'LSH',
        'vals': best_params_lsh[:-1],
        'window': best_params_lsh[-1],
        'dataset': normalized_dataset,
        'encoded_dataset': encoded_dataset,
    }

    compute_silhouette(model, config)

def run_kmeans_cube(model):
    normalized_dataset, encoded_dataset = model_to_files[model][0], model_to_files[model][2]

    config = {
        'model': b'CUBE',
        'vals': best_params_hypercube[:-1],
        'window': best_params_hypercube[-1],
        'dataset': normalized_dataset,
        'encoded_dataset': encoded_dataset,
    }

    compute_silhouette(model, config)

The following cell runs 4 threads in parallel at a time. When one finishes, next one is started. There is no need for locking as Python's code is always executed in a single thread because of GIL (Global Interpreter Lock). C++ code is executed in its own thread (and CPU for our case) and is not affected by GIL as it is released from ctypes module.

In [ ]:
pool = ThreadPool(processes=4)

for model in models:
    pool.apply_async(run_kmeans_classic, (model,))
    pool.apply_async(run_kmeans_lsh, (model,))
    pool.apply_async(run_kmeans_cube, (model,))

pool.close()
pool.join()

In [ ]:
col_models, col_methods, col_stotal_lat_init, col_obj_func = [], [], [], []

for row in rows:
    model, method, stotal_lat_init, obj_func = row

    col_models.append(model)
    col_methods.append(method)
    col_stotal_lat_init.append(stotal_lat_init)
    col_obj_func.append(obj_func)

col_dict = {'model': col_models, 'method': col_methods, 'silhouette total (latent to initial)': col_stotal_lat_init, 'objective function': col_obj_func}

df = pandas.DataFrame(data=col_dict)
df